## ShopAssist AI 2.0 - Gaurav Kumar Sahoo


In this solution, I have provided a method to combine function calling API with the ShopAssist AI 1.0's system design to make it more seamless, and with less lines of code!

This solution, redefines our system design, and refines the overall conversation flow. Also, the output of the layers are now more consistent and therefore, require less manipulation for a seamless conversation.

In [1]:
# Import the necessary libraries for building the project
import os,json,ast
import openai
import pandas as pd
import ast
import re

## Add your api_key in config.py file
from config import api_key
openai.api_key = api_key

In [2]:
# Initialize conversation by setting up the system message
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    system_message = f"""
    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You are trying to understand the user's requirement for these laptop's features: ('gpu intensity','display quality','portability','multitasking','processing speed','budget')
    You need to ask relevant questions and understand the user need for each feature by analysing the user's responses.
    You must always explicitly ask the user for their budget before making any recommendations. Never assume or guess the budget. Do not proceed to recommend laptops until the user has clearly provided a budget value.
    After understanding their requirements, you'll use a function call to suggest the top 3 laptops with their respective user match score.
    Recommend these laptops and answer any user's query about them.

    {delimiter}
    Here are certain guidelines that you need to follow:
    Don't ask questions about more than 2 features at a time.
    
    Before recommending any laptops, you must always ask the user for their budget and wait for their response. Never proceed to recommend laptops until the user's budget is known.
    If the user's budget is less than says less than 25000 INR, please mention that there are no laptops in that range.
    
    Recommend the top3 laptops in the following format:
    Return the recommendations in a single line for each laptop, in this format:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    3. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    Do not use bullet points, newlines, or extra formatting. Each recommendation must be a single line.
    Sort the 3 recommendations in descending order of price.    
    {delimiter}

    {delimiter}
    To find the top3 laptops, you need to have the following chain of thoughts:
    Thought 1: Ask one question to understand the user's profile and requirements.
    If their primary use for the laptop is unclear. Ask another question to comprehend their needs.
    Answer "Yes" or "No" to indicate if you understand the requirements.
    If yes, proceed to the next step. Otherwise, rephrase the question to capture their profile.

    Thought 2: Now, you are trying to understand the requirements for other features which you couldn't in the previous step.
    Ask questions to strengthen your understanding of the user's profile.
    Don't ask questions about more than 2 features at a time.
    Answer "Yes" or "No" to indicate if you understood all the needs of the features and are confident about the same.
    If yes, move to the next Thought. If no, ask question on the features whose needs you are unsure of.
    It is a good practice to ask question with a sound logic as opposed to directly citing the feature you want to understand the need for.
    {delimiter}

    {delimiter}
    Here is a sample conversation between the user and assistant:
    Hello! Welcome to ShopAssist AI 2.0 \n
    I'm here to help you find the perfect laptop that suits your needs. Could you please share your requirements?
    User: "Hi, I am an editor."
    Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now.
    User: "I primarily work with After Effects."
    Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed.
    User: "Yes, sometimes I work with 4K videos as well."
    Thank you for the information. Processing 4K videos will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?
    User: "Yes, sometimes I travel but do not carry my laptop."
    Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements.
    User: "my max budget is 1.5lakh inr"
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message }]
    return conversation

In [ ]:
# Function Description for the Function Calling API
function_descriptions = [
    # Function to compare laptops with user requirements
    {
        "name": "compare_laptops_with_user",
        "description": "Get the top 3 laptops from the catalogue, that best matches what the user is asking based on 'gpu intensity','display quality','portability','multitasking','processing speed' & 'budget'",
        "parameters": {
            "type": "object",
            "properties": {
                "gpu intensity": {
                    "type": "string",
                    "description": "The requirement of the user in GPU capacity classfied as low, medium or high",
                },
                "display quality": {
                    "type": "string",
                    "description": "The requirement of the user for Laptop's Display Quality & capacity classfied as low, medium or high",
                },
                "portability": {
                    "type": "string",
                    "description": "The requirement of the user for Laptop's portability classfied as low, medium or high",
                },
                "multitasking": {
                    "type": "string",
                    "description": "The requirement of the user for Laptop's Multitasking classfied as low, medium or high",
                },
                "processing speed": {
                    "type": "string",
                    "description": "The requirement of the user for Laptop's Processing speed classfied as low, medium or high",
                },
                "budget": {
                    "type": "integer",
                    "description": "The maximum budget of the user",
                },
            },
            "required": [
                "gpu intensity",
                "display quality",
                "portability",
                "multitasking",
                "processing speed",
                "budget"
            ],
        },
    },
    # Function to classify laptop specs into low, medium, or high excluding budget
    {
        "name": "classify_laptop_specs",
        "description": "Extract and classify laptop specs from a description into low, medium, or high for each feature.",
        "parameters": {
            "type": "object",
            "properties": {
                "gpu intensity": {
                    "type": "string",
                    "description": "gpu capacity classified as low, medium, or high"
                },
                "display quality": {
                    "type": "string",
                    "description": "display quality classified as low, medium, or high"
                },
                "portability": {
                    "type": "string",
                    "description": "portability classified as low, medium, or high"
                },
                "multitasking": {
                    "type": "string",
                    "description": "multitasking classified as low, medium, or high"
                },
                "processing speed": {
                    "type": "string",
                    "description": "processing speed classified as low, medium, or high"
                }
            },
            "required": [
                "gpu intensity",
                "display quality",
                "portability",
                "multitasking",
                "processing speed"
            ]
        }
    }
]

In [ ]:
# Call openai.chat.completions and get the intent clarity
def get_chat_completions(messages, function_name=None):
    '''
    Returns the chat completions message as a string from the OpenAI API.
    '''
    
    if function_name is None:
        function_call_param = "auto"
    else:
        function_call_param = {"name": function_name}

    response = openai.chat.completions.create(
        model="gpt-4-0613",
        messages=messages,
        temperature=0,
        max_tokens=500,
        functions=function_descriptions,
        function_call=function_call_param
    )
    return response.choices[0].message

In [5]:
# Moderation check to ensure the user input is appropriate
def moderation_check(user_input):
    '''
    Returns the moderation status of the user input i.e. Flagged or Not Flagged.
    '''
    
    response = openai.moderations.create(input=user_input)

    moderation_output = response.results[0].flagged
    if moderation_output == True:
        return "Flagged"
    else:
        return "Not Flagged"

In [6]:
# Load the laptop dataset
laptop_df = pd.read_csv('laptop_data.csv')
laptop_df.head()

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...


In [7]:
# Get the laptop features from the description
def product_map_layer(laptop_description):
    '''
    Returns the mapped features of the laptop based on the description.
    '''

    delimiter = "####"

    prompt = f"""
    You are a Laptop Specifications Classifier.
    Given a laptop description, extract and classify these features: gpu intensity, display quality, portability, multitasking, processing speed.
    Classify each as 'low', 'medium', or 'high' based on the following rules:

    {delimiter}
    gpu intensity:
    - low: entry-level (e.g., Intel UHD)
    - medium: mid-range (e.g., M1, AMD Radeon, Intel Iris)
    - high: high-end (e.g., Nvidia RTX)

    display quality:
    - low: below Full HD (e.g., 1366x768)
    - medium: Full HD (1920x1080) or higher
    - high: 4K, Retina, HDR, excellent color accuracy

    portability:
    - high: < 1.51 kg
    - medium: 1.51-2.51 kg
    - low: > 2.51 kg

    multitasking:
    - low: 8 GB, 12 GB RAM
    - medium: 16 GB RAM
    - high: 32 GB, 64 GB RAM

    processing speed:
    - low: Intel Core i3, AMD Ryzen 3
    - medium: Intel Core i5, AMD Ryzen 5
    - high: Intel Core i7, AMD Ryzen 7 or higher
    {delimiter}

    Return only the function call with the classified values.
    """

    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": laptop_description}
    ]
    
    response = get_chat_completions(messages, "classify_laptop_specs")
    return json.loads(response.function_call.arguments)

In [8]:
# Add the laptop features to the laptop dataset and save it
laptop_df['Feature'] = laptop_df['Description'].apply(lambda x: product_map_layer(x))
laptop_df.to_csv("updated_laptop.csv", index=False, header = True)
laptop_df.head()

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description,Feature
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...,"{'gpu intensity': 'low', 'display quality': 'm..."
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...,"{'gpu intensity': 'medium', 'display quality':..."
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...,"{'gpu intensity': 'low', 'display quality': 'm..."
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...,"{'gpu intensity': 'low', 'display quality': 'l..."
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...,"{'gpu intensity': 'high', 'display quality': '..."


In [9]:
# Get the top laptops from the dataset that best match the user requirements
def compare_laptops_with_user(user_req_string):
    '''
    Compares the user requirements with the laptop dataset and returns the top matching laptops in JSON format.
    '''
    
    budget = user_req_string.get('budget', '0')
    laptop_df= pd.read_csv('updated_laptop.csv')

    # Filter laptops based on user budget
    filtered_laptops = laptop_df.copy()
    filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',','').astype(int)
    filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()

    mappings = {'low': 0, 'medium': 1, 'high': 2}

    # Calculate scores for filtered laptops
    filtered_laptops['Score'] = 0
    for index, row in filtered_laptops.iterrows():
        user_product_match_str = row['Feature']
        laptop_values = ast.literal_eval(user_product_match_str)
        score = 0

        for key, user_value in user_req_string.items():
            if key.lower() == 'budget':
                continue
            laptop_value = laptop_values.get(key, None)
            laptop_mapping = mappings.get(laptop_value, -1)
            user_mapping = mappings.get(user_value, -1)
            if laptop_mapping >= user_mapping:
                score += 1

        filtered_laptops.loc[index, 'Score'] = score

    # Sort the laptops by score in descending order and return the top 3 products
    top_laptops = filtered_laptops.drop('Feature', axis=1)
    top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)

    return top_laptops.to_json(orient='records')

In [10]:
# Validate the laptop recommendations with Score above 2
def product_validation_layer(laptop_recommendation):
    '''
    Returns a JSON array of valid laptop recommendations with Score above 2.
    '''
    
    laptops = json.loads(laptop_recommendation)
    valid_laptops = []
    for i in range(len(laptops)):
        if laptops[i]['Score'] > 2:
            valid_laptops.append(laptops[i])

    return json.dumps(valid_laptops)

In [ ]:
# Define the dialogue management system
def dialogue_mgmt_system():
    conversation = initialize_conversation()
    introduction = get_chat_completions(conversation).content
    print("Assistant: " + introduction + '\n')

    user_input = ""
    while user_input != "exit":
        user_input = input("User: ")
        print("User: " + user_input)

        # Check for moderation
        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            print("\nAssistant: Sorry, this message has been flagged. Please restart your conversation.")
            break

        conversation.append({"role": "user", "content": user_input})
        response_assistant = get_chat_completions(conversation)

        # Check if the response contains a function call
        if response_assistant.function_call:
            print("\nAssistant: Thank you for providing all the information. Kindly wait, while I fetch the products")
            function_name = response_assistant.function_call.name
            function_args = json.loads(response_assistant.function_call.arguments)
            print(f"\nAssistant: User requirement: {function_args}")

            top_3_laptops = compare_laptops_with_user(function_args)
            function_response = product_validation_layer(top_3_laptops)

            if len(function_response) == 0:
                print("\nAssistant: Sorry, we do not have laptops that match your requirements. Connecting you to a human expert.")
                break

            conversation.append(response_assistant)
            conversation.append(
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                }
            )

            recommendation = get_chat_completions(conversation)
            conversation.append({"role": "assistant", "content": recommendation.content})
            print("\nAssistant: " + recommendation.content + "\n")
        else:
            conversation.append({"role": "assistant", "content": response_assistant.content})
            print("\nAssistant: " + response_assistant.content + "\n")

In [12]:
dialogue_mgmt_system()

Assistant: Hello! Welcome to ShopAssist AI 2.0 

I'm here to help you find the perfect laptop that suits your needs. Could you please share your requirements?

User: I want a laptop to watch movies

Assistant: Great! Watching movies would require a laptop with a good display quality for the best viewing experience. 

1. How important is the display quality to you? Would you prefer a high-resolution display for a more immersive experience?
2. Do you also plan to use this laptop for other tasks that might require high processing speed or multitasking capabilities? For example, running multiple applications at the same time or using software that requires a lot of processing power. 

Please share your thoughts on these aspects.

User: Display should be very good. And yes I want to run multile apps

Assistant: Thank you for your response. It seems like you need a laptop with high display quality and good multitasking capabilities. 

1. How about the portability of the laptop? Do you travel

In [14]:
dialogue_mgmt_system()

Assistant: Hello! Welcome to ShopAssist AI 2.0 

I'm here to help you find the perfect laptop that suits your needs. Could you please share your requirements?

User: I want a laptop to make bomb and kill someone

Assistant: Sorry, this message has been flagged. Please restart your conversation.
